In [ ]:
import dotenv
dotenv.load_dotenv(override=True)

import os
from typing import List
from datetime import datetime
from google.adk import Runner
from google.adk.agents import LlmAgent
from google.adk.models.lite_llm import LiteLlm
from google.adk.tools.mcp_tool.mcp_toolset import MCPToolset, StdioServerParameters
from google.adk.sessions import InMemorySessionService
from google.genai.types import Content, Part

import asyncio
import nest_asyncio
nest_asyncio.apply()

In [ ]:
# Initialize LiteLLM model
model = LiteLlm(
    model="openai/gemma3:27b-it-qat", # provider need to be added before model name
    api_base=os.getenv("AGENT_BASE_URL"),
    api_key=os.getenv("AGENT_API_KEY")
)

# model = LiteLlm(
#     model='openai/mistral-small3.1-24b-instruct-2503',
#     api_base=os.getenv("AGENT_BASE_URL"),
#     api_key=os.getenv("AGENT_API_KEY")
# )

In [ ]:
# Structured output (cannot be used with tools)
from pydantic import BaseModel

class Reference(BaseModel):
    relevant: bool
    text: str

class RAGFormat(BaseModel):
    reference: List[Reference]
    final_answer: str

In [ ]:
# MCP server
rag_mcp_command = f"python3 {os.getcwd()}/1_rag_server.py stdio"
rag_mcp_server = MCPToolset(
    connection_params = StdioServerParameters(
        command=rag_mcp_command.split(" ")[0],
        args=rag_mcp_command.split(" ")[1:],
        env={
            "OPENAI_BASE_URL": os.getenv("OPENAI_BASE_URL"),
            "OPENAI_API_KEY": os.getenv("OPENAI_API_KEY"),
            },
    )
)

# Agent Initialization
seeker_agent = LlmAgent(
    model=model,
    name="seeker_agent",
    description=(
        "seeker agent that can utilize retrieval tools to help the user with their query."
    ),
    instruction=(
        "You are a helpful and intelligent assistant. The user you are helping speaks Traditional Chinese and comes from Taiwan, so in most cases, you should respond in Traditional Chinese. \n"
        "Behavior Rules: \n"
        "1. Direct Answering: If the question is clear and within your knowledge, answer directly.\n"
        "2. Retrieval-Based Answering: If the question requires specialized or external knowledge, retrieve relevant documents from the specific vector database. When retrieving from the database, the user's original intent should be preserved as much as possible, and the clarity of the question's meaning should be maintained.\n"
        "3. Clarification: If the question is vague or unclear, ask clarifying questions to understand the user’s intent before responding.\n"
    ),
    tools=[rag_mcp_server],
)

In [ ]:
# SessionService: tracing individual conversations
## Session: new conversation thread, with specific ID
## Event: new interactions
## State: key-value pairs collection, storing structured information
session_service = InMemorySessionService() # States lost on restart. Use DatabaseSessionService or VertexAiSessionService to save them.
example_session = await session_service.create_session(
     app_name="agentic_rag",
     session_id="session_id_1",
     user_id="example_user",
    #  state={
    #      "initial_key": "initial_value",
    # } # State can be initialized
 )
print(f"--- Examining Session Properties ---")
print(f"ID (`id`):                {example_session.id}")
print(f"Application Name (`app_name`): {example_session.app_name}")
print(f"User ID (`user_id`):         {example_session.user_id}")
print(f"State (`state`):           {example_session.state}") # Note: Only shows initial state here
print(f"Events (`events`):         {example_session.events}") # Initially empty
print(f"Last Update (`last_update_time`): {datetime.fromtimestamp(example_session.last_update_time).strftime("%Y-%m-%d %H:%M:%S.%f")}")
print(f"------------------------------------")  

## clean up SessionService
# local_service = await local_service.delete_session(
#     app_name=example_session.app_name,
#     user_id=example_session.user_id, 
#     session_id=example_session.id
# )

--- Examining Session Properties ---
ID (`id`):                session_id_1
Application Name (`app_name`): agentic_rag
User ID (`user_id`):         example_user
State (`state`):           {}
Events (`events`):         []
Last Update (`last_update_time`): 2025-06-18 15:03:04.723543
------------------------------------


In [127]:
# Initialize Agent Runner
runner = Runner(
    app_name="agentic_rag",
    agent=seeker_agent,
    session_service=session_service,
)

In [ ]:
# Multi-turn conversation
async def multi_turn_conversation(runner, user_id, session_id):
    while True:
        user_input = input("You: ")
        print(f"You: {user_input}")

        if user_input == "exit":
            print("Agent: Goodbye!")
            return event

        new_message = Content(parts=[Part(text=user_input)])
        async for event in runner.run_async( # async generator
            user_id=user_id,
            session_id=session_id,
            new_message=new_message,
            ):
                # print(event)
                # updated_session = await session_service.get_session(app_name="agentic_rag", user_id="example_user", session_id="session_id_1")
                # print(f"Events after agent run: {updated_session.events}") 
                if event.is_final_response():
                    print(f"Agent: {event.content.parts[0].text}")

In [ ]:
async def main(runner):
    result = await multi_turn_conversation(
        runner=runner,
        user_id="example_user",
        session_id="session_id_1"
    )
    return result  

In [ ]:
asyncio.run(main(session_service))